In [1]:
%load_ext lab_black

In [2]:
import time
import itertools
import warnings

import cvxpy as cp
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
from scipy import sparse as spr
from scipy import special

np.random.seed(1)
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Question 2: Maximum likelihood prediction of team ability
A set of $n$ teams compete in a tournament. We model each team's ability by a number  $a_j \in [0,1]$,  $j=1,\ldots, n$ . When teams $j$ and $k$ play each other, the probability that team $j$ wins is equal to $\mathbf{prob}(a_j-a_k+v>0)$, where $v \sim \mathcal N(0,\sigma^2)$.

You are given the outcome of $m$ past games. These are organized as

$$
(j^{(i)},k^{(i)},y^{(i)}), \quad i = 1,\ldots,m,
$$

meaning that game $i$ was played between teams $j^{(i)}$ and $k^{(i)}$; $y^{(i)}=1$ means that team $j^{(i)}$ won, while $y^{(i)}=-1$ means that team $k^{(i)}$ won. (We assume there are no ties.)

Formulate the problem of finding the maximum likelihood estimate of team abilities, $\hat a \in \mathbf{R}^n$, given the outcomes, as a convex optimization problem. You will find the *game incidence matrix* $A\in \mathbf{R}^{m \times n}$, defined as

$$
A_{il} = \left\{\begin{array}{rl}y^{(i)} & l = j^{(i)}\\-y^{(i)} & l = k^{(i)}\\0 & \mbox{otherwise},\end{array}\right.
$$

useful.

The prior constraints $\hat a_i \in [0,1]$ should be included in the problem formulation. Also, we note that if a constant is added to all team abilities, there is no change in the probabilities of game outcomes. This means that  𝑎̂   is determined only up to a constant, like a potential. But this doesn't affect the ML estimation problem, or any subsequent predictions made using the estimated parameters.

Find $\hat a$ for the team data given in `team_data.m`, in the matrix train. (This matrix gives the outcomes for a tournament in which each team plays each other team once.) You may find the CVX function `log_normcdf` helpful for this problem.

You can form $A$ using the commands
```
A = sparse(1:m,train(:,1),train(:,3),m,n) + ...
    sparse(1:m,train(:,2),-train(:,3),m,n);
```
Finally, use the maximum likelihood estimate  𝑎̂   to predict the outcomes of next year's tournament games, given in the matrix test, using $\hat y^{(i)} = \mathbf{sign}(\hat a_{j^{(i)}} - \hat a_{k^{(i)}})$. Compare these predictions with the actual outcomes, given in the third column of test. What is the fraction of correctly predicted outcomes? (If half of the outcomes are correctly predicted, the answer is 0.5 and not 50.)

## Solution
First, we need to construct the likelihood function. For the observed data, we know that $a_j-a_k +v \sim \mathcal{N}(a_j-a_k, \sigma^2)$ and that $y\cdot(a_j-a_k +v) > 0$, this implies that:

$$
\begin{align}
\mathbb{P}(y=1) &= \mathbb{P}(a_{j^{(i)}}-a_{k^{(i)}} + v > 0) \\
&= 1 - \Phi\left(\frac{0-(a_{j^{(i)}}-a_{k^{(i)}})} {\sigma}\right) \\
&= \Phi\left(\frac{a_{j^{(i)}}-a_{k^{(i)}}} {\sigma}\right) \\
\mathbb{P}(y=-1) &= \mathbb{P}(a_{j^{(i)}}-a_{k^{(i)}} + v < 0) \\
&= \Phi\left(\frac{0-(a_{j^{(i)}}-a_{k^{(i)}})} {\sigma}\right) \\
&= \Phi\left(\frac{-(a_{j^{(i)}}-a_{k^{(i)}})} {\sigma}\right) 
\end{align}
$$
$$
\boxed{\therefore \mathbb{P}(y=y_i) = \Phi\left(\frac{y_i\cdot(a_{j^{(i)}}-a_{k^{(i)}})} {\sigma}\right) }
$$

Thus the likelihood function is:
$$
p_a(y) = \prod_{i=1}^m  \Phi\left(\frac{y^{(i)}\cdot(a_{j^{(i)}}-a_{k^{(i)}})} {\sigma} \right)
$$
Which gives the log-likelihood:
$$
\boxed{l(a, y) = \sum_{i=1}^m \ln\left(\Phi\left(\frac{y^{(i)}\cdot(a_{j^{(i)}}-a_{k^{(i)}})} {\sigma}\right)\right)}
$$

In [3]:
def create_game_incidence_matrix(data, num_teams):
    num_games = data.shape[0]
    gim = np.zeros((num_games, num_teams))
    for p, r in enumerate(data):
        j, k, y = r
        gim[p, j - 1] = y
        gim[p, k - 1] = -y
    gim_sp = spr.csc_matrix(gim)
    return gim_sp


def get_team_data():
    n = 10
    m = 45
    m_test = 45
    sigma = 0.250
    train = np.array(
        [
            [1, 2, 1],
            [1, 3, 1],
            [1, 4, 1],
            [1, 5, 1],
            [1, 6, 1],
            [1, 7, 1],
            [1, 8, 1],
            [1, 9, 1],
            [1, 10, 1],
            [2, 3, -1],
            [2, 4, -1],
            [2, 5, -1],
            [2, 6, -1],
            [2, 7, -1],
            [2, 8, -1],
            [2, 9, -1],
            [2, 10, -1],
            [3, 4, 1],
            [3, 5, -1],
            [3, 6, -1],
            [3, 7, 1],
            [3, 8, 1],
            [3, 9, 1],
            [3, 10, 1],
            [4, 5, -1],
            [4, 6, -1],
            [4, 7, 1],
            [4, 8, 1],
            [4, 9, -1],
            [4, 10, -1],
            [5, 6, 1],
            [5, 7, 1],
            [5, 8, 1],
            [5, 9, -1],
            [5, 10, 1],
            [6, 7, 1],
            [6, 8, 1],
            [6, 9, -1],
            [6, 10, -1],
            [7, 8, 1],
            [7, 9, 1],
            [7, 10, -1],
            [8, 9, -1],
            [8, 10, -1],
            [9, 10, 1],
        ]
    )

    test = np.array(
        [
            [1, 2, 1],
            [1, 3, 1],
            [1, 4, 1],
            [1, 5, 1],
            [1, 6, 1],
            [1, 7, 1],
            [1, 8, 1],
            [1, 9, 1],
            [1, 10, 1],
            [2, 3, -1],
            [2, 4, 1],
            [2, 5, -1],
            [2, 6, -1],
            [2, 7, -1],
            [2, 8, 1],
            [2, 9, -1],
            [2, 10, -1],
            [3, 4, 1],
            [3, 5, -1],
            [3, 6, 1],
            [3, 7, 1],
            [3, 8, 1],
            [3, 9, -1],
            [3, 10, 1],
            [4, 5, -1],
            [4, 6, -1],
            [4, 7, -1],
            [4, 8, 1],
            [4, 9, -1],
            [4, 10, -1],
            [5, 6, -1],
            [5, 7, 1],
            [5, 8, 1],
            [5, 9, 1],
            [5, 10, 1],
            [6, 7, 1],
            [6, 8, 1],
            [6, 9, 1],
            [6, 10, 1],
            [7, 8, 1],
            [7, 9, -1],
            [7, 10, 1],
            [8, 9, -1],
            [8, 10, -1],
            [9, 10, 1],
        ]
    )

    A_train = create_game_incidence_matrix(train, n)
    A_test = create_game_incidence_matrix(test, n)
    return n, m, sigma, m_test, train, test, A_train, A_test

In [4]:
n, m, sigma, m_test, train, test, A_train, A_test = get_team_data()

In [5]:
def cvx_log_noramcdf(x: cp.Variable):
    cdf = 0.5 * (1 + np.erf(x_))
    return cdf

### Polynomial Approximation of `erf`
$$
\begin{aligned}
\operatorname {erf} (x) &={\begin{cases}1-\tau &x\geq 0\\\tau -1&x<0\end{cases}} \\
\tau &:=t\cdot \exp \left(-x^{2}-1.26551223+1.00002368t+0.37409196t^{2}+0.09678418t^{3}-0.18628806t^{4}\right.\\
&\left.\qquad \qquad \qquad +0.27886807t^{5}-1.13520398t^{6}+1.48851587t^{7}-0.82215223t^{8}+0.17087277t^{9}\right) \\
t&:={\frac {1}{1+0.5|x|}}
\end{aligned}
$$

In [18]:
# log_normcdf is not in cvxpy and not trivial to supprt it.
def erf(x):
    coefs = (
        -1.26551223,
        1.00002368,
        0.37409196,
        0.09678418,
        -0.18628806,
        0.27886807,
        -1.13520398,
        1.48851587,
        -0.82215223,
        0.17087277,
    )
    t = 1 / (1 + cp.abs(x) / 2)
    τ = cp.multiply(
        t,
        cp.exp(
            -cp.power(x, 2)
            + cp.sum(
                [cp.multiply(coef, cp.power(t, p)) for p, coef in enumerate(coefs)]
            )
        ),
    )
    return 1 - τ  # if x >= 0 else τ - 1

In [23]:
a = cp.Variable(n)
z = A_train @ a / sigma / np.sqrt(2)
constraints = [a >= 0, a <= 1]
obj = cp.Maximize(cp.sum(cp.log((1 + erf(z)) / 2)))
prob = cp.Problem(obj, constraints)

In [25]:
# prob.solve()

## Question 3: Allocation of interdiction effort
A smuggler moves along a directed acyclic graph with $m$ edges and $m$ nodes, from a source node (which we take as node  1) to a destination node (which we take as node $n$), along some (directed) path. Each edge $k$ has a detection failure probability $p_k$, which is the probability that the smuggler passes over that edge undetected. The detection events on the edges are independent, so the probability that the smuggler makes it to the destination node undetected is $\prod_{j \in \mathcal P} p_j$, where $\mathcal P~\subset~\{1,\ldots, m\}$ is (the set of edges on) the smuggler's path. We assume that the smuggler knows the detection failure probabilities and will take a path that maximizes the probability of making it to the destination node undetected. We let $P^\max$ denote this maximum probability (over paths). (Note that this is a function of the edge detection failure probabilities.)

The edge detection failure probability on an edge depends on how much interdiction resources are allocated to the edge. Here we will use a very simple model, with $x_j \in \mathbf{R}_+$ denoting the effort (say, yearly budget) allocated to edge $j$, with associated detection failure probability $p_j = e^{-a_j x_j}$, where $a_j \in \mathbf{R}_{++}$ are given. The constraints on $x$ are a maximum for each edge, $x \preceq x^\mathrm{max}$, and a total budget constraint, $\mathbf{1}^Tx \leq B$.

Pose the problem of choosing the interdiction effort vector $x \in \mathbf{R}^m$ that minimizes $P^\max$, subject to the constraints, as a convex optimization problem. Your formulation should not enumerate all possible paths (in the objective or constraints). Hint. For each node $i$, let $P_i$ denote the maximum of $\prod_{k\in \mathcal P}p_k$ over all paths $\mathcal P$ from the source node 1 to node $i$ (so $P^\max = P_n$).

Carry out your method on the problem instance given in `interdict_alloc_data.m`. The data file contains the data  $a,x^\max,B$, and the graph incidence matrix $A \in \mathbf{R}^{n \times m}$, where

$$
A_{ij} = \left\{ \begin{array}{ll} -1 & \mbox{if edge \(j\) leaves node \(i\)}\\+1 & \mbox{if edge \(j\) enters node \(i\)}\\0 & \mbox{otherwise}. \end{array}\right.
$$

Give $P^{\mathrm{max}\star}$, the optimal value of $P^\max$, and compare it to the value of $p^\max$ obtained with uniform allocation of resources, i.e., with $x= (B/m) \mathbf{1}$.

Hint. Given a vector $z \in \mathbf{R}^n$, $A^T z$ is the vector of edge differences: $(A^Tz)_j = z_k - z_l$ if edge $j$ goes from node $l$ to node $k$.

### Solution

What is the value of $P^{\max\star}$? (Write 0.1 for 10%.)

In [54]:
def get_interdict_alloc_data():
    np.random.seed(0)
    n, m = 10, 20
    edges = [
        (0, 1),
        (0, 2),
        (0, 3),
        (1, 2),
        (1, 3),
        (1, 5),
        (2, 4),
        (2, 5),
        (3, 5),
        (3, 6),
        (4, 6),
        (4, 7),
        (5, 6),
        (5, 7),
        (6, 7),
        (6, 8),
        (6, 9),
        (7, 8),
        (7, 9),
        (8, 9),
    ]
    a = 2 * np.random.rand(m)
    x_max = 1 + np.random.rand(m)
    B = m / 2.0
    A = np.zeros((n, m))
    for k, (i, j) in enumerate(edges):
        A[i, k] = -1
        A[j, k] = 1
    return n, m, a, x_max, B, edges, A

In [55]:
# Not equal to MATLAB generated data due to difference in RNGs.
n, m, a, x_max, B, edges, A = get_interdict_alloc_data()

What is the value of $P^\max$ obtained with uniform allocation of resources? (Write 0.1 for 10%.)

In [64]:
x = cp.Variable(m, nonneg=True)
z = cp.Variable(n)

obj = cp.Minimize(z[-1])
constraints = [z[0] == 0, A.T @ z >= -cp.diag(a) @ x, x <= x_max, cp.sum(x) <= B]

prob = cp.Problem(obj, constraints)

In [66]:
opt = prob.solve()
opt

-2.9116454040875612

In [69]:
P_max = np.exp(opt)
P_max

0.054386168982720653

What is the value of $P^\max$ obtained with uniform allocation of resources? (Write 0.1 for 10%.)

In [77]:
prob_unifrom = cp.Problem(prob.objective, constraints + [x == B / m])

In [78]:
opt_unifrom = prob_unifrom.solve()
P_max_uniform = np.exp(opt_unifrom)
P_max_uniform

0.36552401686307545